In [2]:
# start coding here
token = snakemake.params

# PitViper Notebook Report

This notebook was generated automatically by PitViper.

It can be used in two ways:

1. By using the functions already created and present in the following cells.

2. By creating new cells and writing python3 code in them.

The graphs are generated using the python library [Altair](https://altair-viz.github.io/index.html). It is possible to download each graph in SVG format from the drop-down menu at the top right of each graph.

The next cell allows to call the functions already created for the visualization of the results.

In [3]:
import sys
import os
import pandas as pd
import IPython

modules_path = ['workflow/notebooks/', "../../../workflow/notebooks/"]
for module in modules_path:
    module_path = os.path.abspath(os.path.join(module))
    if module_path not in sys.path:
        sys.path.append(module_path)


from functions import * 

import time

alt.renderers.enable('html')

# For Snakemake
switch = True
while os.path.basename(os.getcwd()) != "PitViper":
    if switch:
        switch = False
        %cd ../../
    else:
        %cd ../

print('Working directory: ', os.getcwd())

with open(snakemake.output[0], "w") as out:
    print("Notebook was runned.", file=out)

### rpy2

Next cell load rpy2 which allow to define cells running R code by adding `%%R` at the first line of the cell. 

Note: it's possible to forward some python-specific objects (such as list or pandas dataframe) to R cells using `-i` parameters followed by objects name (`%%R -i token`, token string is then available as an R object).

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R

source("workflow/notebooks/functions_R.R")

## Process data

Next function scan `results/` directory to retrieve all results.

`tools_available` is python dictionnary in which all data are stored in a comprehensive manner.

In [6]:
results_directory, tools_available = setup_step_1(token)

## Mapping Quality Control

If available, mapping quality control metrics will be shown by next function (`show_mapping_qc`).

In [7]:
show_mapping_qc(token)

## Read count distribution

Normalized read count distribution for all replicates will be shown by calling `show_read_count_distribution` function.

In [8]:
alt.data_transformers.disable_max_rows()

show_read_count_distribution(token)

## Principal component analysis

In [9]:
pca_counts(token)

## Tools global results

In [10]:
snake_plot(results_directory, tools_available)

## sgRNA read counts by element

In [11]:
show_sgRNA_counts(token) 

In [12]:
%%R -i token

config_data <- get_data(token)

tsv_file <- config_data$tsv
cts_file <- config_data$cts

mean_counts_by_condition(conditions = "J4,J11,J18,J25",
                         gene = "MYC")

## results by tool and by element

In [13]:
CRISPhieRmix_results(results_directory, tools_available)

In [14]:
GSEA_like_results(results_directory, tools_available)

In [15]:
in_house_method_results(results_directory, tools_available)

In [16]:
MAGeCK_RRA_results(results_directory, tools_available)

In [17]:
MAGeCK_MLE_results(results_directory, tools_available)

In [18]:
BAGEL_results(results_directory, tools_available)

In [19]:
enrichr_plots(tools_available)

## GeneMania

In [20]:
genemania_link_results(tools_available)

## Definition of ranks and occurrences

In next cell, choose which conditions to compare.

In [21]:
TREATMENT = "J25"
CONTROL = "J4"

In [22]:
ranks, occurences = ranking(TREATMENT, CONTROL, token, tools_available)

## Venn diagram

In [23]:
%%R -i occurences,CONTROL,TREATMENT

venn_diagram(occurences, TREATMENT, CONTROL)

## Essentiality prioritisation using RobustRankAggregation

In this section, the `RobustRankAggreg` (RRA) method is used to integrate ranks derived from the results calculated by each selected method.

In [24]:
%%R -i ranks

res <- RobustRankAggregate(ranks)

res %>% filter(Score < 0.05)

## depmap integration with "Essentiality prioritisation" results

In [25]:
%%R

library(depmap)
library("ExperimentHub")
library(ggplot2)

## create ExperimentHub query object
eh <- ExperimentHub()

rnai <- eh[["EH2260"]]

rnai

In [26]:
%%R

query.gene <- "MYC"
query.cell_line <- "KASUMI2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE"
query.cell_line_name <- "KASUMI2"

essential.genes <- res %>%
    filter(Score < 0.05) %>%
    pull(Name)


rnai.filtered <- rnai %>%
    filter(cell_line == query.cell_line) %>%
    mutate(essential = ifelse(gene_name %in% essential.genes, "essential", "non-essential")) %>%
    filter(!is.na(dependency))


gene.dependency <- rnai.filtered %>%
    filter(gene_name == query.gene) %>%
    pull(dependency)


rnai.filtered %>%
    ggplot(aes(x=dependency, fill=essential)) + 
    geom_density(alpha=0.4) + 
    theme_classic() +
    scale_fill_manual(values=c("#E69F00", "#999999")) +
    geom_vline(xintercept = 0) +
    geom_rug(data = subset(rnai.filtered, essential == "essential"), col="#E69F00", alpha=0.5, sides = "t") +
    geom_rug(data = subset(rnai.filtered, essential == "non-essential"), col="#999999", alpha=0.5, outside = TRUE, sides = "t") +
    coord_cartesian(clip = "off") +
    geom_vline(xintercept = gene.dependency, linetype="dashed", color = "red", size=1) +
    ggtitle(paste0(query.cell_line_name, "dependencies (", query.gene," highlighted)")) +
    theme(plot.title = element_text(vjust = 4),
          plot.margin = margin(10, 10, 10, 10))


In [27]:
%%R

library(ggridges)
library(tidyr)
library(forcats)

rnai.filtered <- rnai %>%
    filter(grepl("HAEMATOPOIETIC_AND_LYMPHOID_TISSUE", cell_line)) %>%
    mutate(essential = ifelse(gene_name %in% essential.genes, "essential", "non-essential")) %>%
    filter(!is.na(dependency))


rnai.filtered %>%
    mutate(cell_line = fct_reorder(.f = cell_line, .x = -dependency, .fun = median)) %>%
    filter(essential == "essential") %>%
    ggplot(aes(x = `dependency`, y = cell_line, fill = stat(x))) +
        geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01, quantile_lines = TRUE, quantiles = 2) +
        scale_fill_viridis_c() +
        geom_vline(xintercept=-0.5, color = "red", size=1) +
        geom_vline(xintercept=0, color = "black", size=1, linetype="dashed") +
        theme_classic()


In [1]:
tools = list(tools_available.keys())


def plot_chart(data, tool, condition, file, x, y, method, column_filter='', cutoff=0, greater=False, element=''):
    source = data[tool][condition][file]
    if method == 'Cut-off':
        if (source[column_filter].dtype in [np.float64, np.int64]):
            pass_value = "%s_<_%s" % (column_filter,cutoff)
            fail_value = "%s_>=_%s" % (column_filter,cutoff)
            source.loc[source[column_filter] < cutoff, 'filter'] = pass_value
            source.loc[source[column_filter] >= cutoff, 'filter'] = fail_value
        else:
            source['filter'] = 'pass'
            print('Choose a column of integer or float type.')
            
    chart = alt.Chart(source).mark_circle().encode(
        x=x,
        y=y,
        tooltip = list(source.columns),
        color='filter'
    ).interactive()
    display(chart)



@interact(
    tool = widgets.Dropdown(options=tools,value=tools[0],description='Tool:',disabled=False))
def form(tool):
    conditions = list(tools_available[tool].keys())
    @interact(
        condition = widgets.Dropdown(options=conditions,value=conditions[0],description='Condition:',disabled=False))
    def form(condition):
        files = list(tools_available[tool][condition].keys())
        @interact(file = widgets.Dropdown(options=files,value=files[0],description='File:',disabled=False))
        def form(file):
            columns = tools_available[tool][condition][file].columns
            @interact(x = widgets.Dropdown(options=columns,value=columns[0],description='x:',disabled=False),
                      y = widgets.Dropdown(options=columns,value=columns[1],description='y:',disabled=False))
            def form(x , y):
                @interact(method = widgets.ToggleButtons(options=['Cut-off', 'Variable'],value='Cut-off', description='Method:'))
                def form(method):
                    if method == 'Cut-off':
                        @interact(column_filter = widgets.Dropdown(options=columns,value=columns[0],description='Column:',disabled=False),
                                  cutoff = widgets.FloatText(value=0.0,description='Cut-off:'),
                                  greater = widgets.Checkbox(value=False,description='Greater?',disabled=False,indent=True),
                                  pass_color = widgets.ColorPicker(concise=False,description='Pass color:',value='red',disabled=False),
                                  fail_color = widgets.ColorPicker(concise=False,description='Fail color:',value='gray',disabled=False),
                                  column_element = widgets.Dropdown(options=columns,value=columns[0],description='Column:',disabled=False))
                        def form(column_filter, cutoff, greater, pass_color, fail_color, column_element):
                            first_element = str(tools_available[tool][condition][file][column_element][0])
                            @interact(element = widgets.Text(value=first_element,placeholder='Element:',description='Element:',disabled=False))
                            def form(element):
                                plot_chart(tools_available, tool, condition, file, x, y, method, column_filter, cutoff, greater, element)
                    elif method == 'Variable':
                        pass

NameError: name 'tools_available' is not defined